# Get AIrsenal's match predictions for a set of fixtures

In [1]:
import numpy as np
import pandas as pd
import operator
from airsenal.framework.bpl_interface import get_fitted_team_model, get_training_data
from airsenal.framework.schema import session
from airsenal.framework.utils import get_fixtures_for_gameweek
import matplotlib.pyplot as plt
from airsenal.framework.utils import *



/Users/crangelsmith/opt/anaconda3/envs/airsenalenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
season = "2122"
max_goals = 8

max_week = get_max_gameweek(season=season,dbsession=session)

print (max_week)

38


Pipeline to run predictions at a given week and the future (as variable).

In [3]:

def run_match_prediction(gw, season, max_goals, n_weeks):
    """
    Fit model based on season and gameweek, get fixtures for that gameweek and predict results. 
    Compare predictions with actual results. If n_weeks =0 the prediction is done only for gameweek (gw). 
    If n_weeks>0 predictions are done to the future but with data up to gameweek (gw). 
    Data of prediction and results returned as dataframe for evaluation of the model.
    
    Parameters
    ----------
    gw: int 
        Gameweek until which data is used in training (non inclusive)
    season: str
        Season of game
    max_goal: int
        Maximum number of goals to get probabilites
    n_weeks: int
        Number of weeks to predict in the future.
        

    Returns
    -------
    DataFrame with prediction, results and evaluation statistics.
    
        
    """
        
    # fit model
    model = get_fitted_team_model(season, gw, session)
    
    # get fixtures for gameweek 
    fixtures = get_fixtures_for_gameweek(gw, season, session)
    
    # get fixtures for futue weeks.
    
    if n_weeks >0:
        for week in range(gw,gw+n_weeks):
            fixtures.extend(get_fixtures_for_gameweek(week, season, session))
        
    
    # get outcome probabilities for fixtures
    outcome_probs = [
    model.predict_outcome_proba(f.home_team, f.away_team) for f in fixtures
    ]
    

    # get most likely outcome based on probability.
    most_likely_outcome = []
    for o in outcome_probs:
        if (o["home_win"] >= o["away_win"]) and (o["home_win"] >= o["draw"]):
            most_likely_outcome.append("home_win")
        elif (o["away_win"] > o["home_win"]) and (o["away_win"] >= o["draw"]):
            most_likely_outcome.append("away_win")
        else:
            most_likely_outcome.append("draw")
            
           
    # get goal probabilities for each fixture
    goal_probs = [
    (
        model.predict_score_n_proba(
            np.arange(max_goals), f.home_team, f.away_team, home=True
        ),
        model.predict_score_n_proba(
            np.arange(max_goals), f.away_team, f.home_team, home=False
        ),
    )
    for f in fixtures
    ]
    # calculate weighted goal outcome for each fixture. 
    mean_goals = [
    (
        round(np.average(np.arange(max_goals), weights=g[0]), 2),
        round(np.average(np.arange(max_goals), weights=g[1]), 2),
    )
    for g in goal_probs
    ]
    
    
    # summarise results and save them to dictionary
    result_prob = []
    for f, o, m in zip(fixtures, most_likely_outcome, mean_goals):
        
        if f.result:
            actual_prob = model.predict_score_proba(
                f.home_team, f.away_team, f.result.home_score, f.result.away_score
            )
            print(f.result, ", PREDICT:", o, m, f", ACTUAL: {float(actual_prob):.1%}")
            
            outcome_probs = model.predict_outcome_proba(f.home_team, f.away_team)
                        
            
            max_predicted_prob = max(outcome_probs.items(), key=operator.itemgetter(1))
            
            n_weeks_ahead = f.gameweek - gw
                            
            results_dict = {'season':f.season,'gameweek':f.gameweek,'home_team':f.home_team, 'away_team':f.away_team,
                           'home_score':f.result.home_score,'away_score':f.result.away_score, 'predicted_home_score':m[0],
                            'predicted_away_score':m[1], 'probability_result':float(actual_prob),'predicted_outcome_prob':max_predicted_prob[1].item(),
                            'predicted_outcome':max_predicted_prob[0],'n_weeks_ahead':n_weeks_ahead}

            
      
            result_prob.append(results_dict)
        
    # turn list of dicts to dataframe and calculate metrics for prediction accuracy.
    result_prob = pd.DataFrame(result_prob)
    result_prob['diff_score_home'] = abs(result_prob['home_score'] - result_prob['predicted_home_score'])
    result_prob['diff_score_away'] = abs(result_prob['away_score'] - result_prob['predicted_away_score'])
    result_prob['diff_score_mean'] = (result_prob['diff_score_home'] + result_prob['diff_score_away'])/2
    result_prob['outcome'] = np.where(result_prob['home_score'] == result_prob['away_score'], 'draw', 'other')
    result_prob['outcome'] = np.where(result_prob['home_score'] > result_prob['away_score'], 'home_win', 'away_win')
    
    result_prob['result_correct'] = np.where(result_prob['outcome'] == result_prob['predicted_outcome'], 1, 0)

    result_prob['home_score_correct'] = np.where(result_prob['home_score'] == result_prob['predicted_home_score'], 1, 0)
    result_prob['away_score_correct'] = np.where(result_prob['away_score'] == result_prob['predicted_away_score'], 1, 0)
    
    result_prob['score_correct'] = (result_prob['home_score_correct'] + result_prob['away_score_correct'])/2
    
    return result_prob


    
     

In [4]:
def make_scatter_error_plot(df,column, colour = 'b'):
    """
    Plot mean and std of a variable by gameweek.
    
    df: DataFrame
        Results dataframe
    column: string
        Column to visualise
    
    """

    # Grouping the DataFrame by 'gameweek' and calculating mean and standard deviation
    grouped = df.groupby('gameweek')[column].agg(['mean', 'std'])

    # Resetting the index to convert the grouped result back to a DataFrame
    grouped = grouped.reset_index()

    # Plotting the mean values
    plt.plot(grouped['gameweek'], grouped['mean'], marker='o',color = colour)

    # Plotting filled error bars
    plt.fill_between(grouped['gameweek'], grouped['mean'] - grouped['std'], grouped['mean'] + grouped['std'], alpha=0.3, color = colour)

    # Adding labels and title to the plot
    plt.xlabel('Game week')
    plt.ylabel(column)
    plt.title(column+' by Gameweek')

    # Displaying the plot
    plt.show()

# Model 1:

Predict 38 weeks of the season with data until week 1

In [5]:
one_to_38_week_prediction = run_match_prediction(1,season, max_goals,max_week)


Fitting team model...


sample: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:18<00:00, 79.39it/s, 127 steps of size 4.15e-02. acc. prob=0.93]


Adding BRE to team model with covariates
2122 GW1 BRE 2 - 0 ARS , PREDICT: away_win (1.08, 1.69) , ACTUAL: 3.6%
2122 GW1 MUN 5 - 1 LEE , PREDICT: home_win (1.83, 0.99) , ACTUAL: 1.0%
2122 GW1 BUR 1 - 2 BHA , PREDICT: home_win (1.18, 0.93) , ACTUAL: 6.2%
2122 GW1 CHE 3 - 0 CRY , PREDICT: home_win (1.98, 0.85) , ACTUAL: 7.6%
2122 GW1 EVE 3 - 1 SOU , PREDICT: home_win (1.6, 1.04) , ACTUAL: 5.0%
2122 GW1 LEI 1 - 0 WOL , PREDICT: home_win (1.57, 1.07) , ACTUAL: 10.6%
2122 GW1 WAT 3 - 2 AVL , PREDICT: home_win (1.31, 1.17) , ACTUAL: 2.1%
2122 GW1 NOR 0 - 3 LIV , PREDICT: away_win (0.88, 2.26) , ACTUAL: 8.4%
2122 GW1 NEW 2 - 4 WHU , PREDICT: away_win (1.34, 1.35) , ACTUAL: 0.8%
2122 GW1 TOT 1 - 0 MCI , PREDICT: away_win (1.46, 1.56) , ACTUAL: 6.5%
2122 GW1 BRE 2 - 0 ARS , PREDICT: away_win (1.08, 1.69) , ACTUAL: 3.6%
2122 GW1 MUN 5 - 1 LEE , PREDICT: home_win (1.83, 0.99) , ACTUAL: 1.0%
2122 GW1 BUR 1 - 2 BHA , PREDICT: home_win (1.18, 0.93) , ACTUAL: 6.2%
2122 GW1 CHE 3 - 0 CRY , PREDICT: ho

2122 GW11 LEE 1 - 1 LEI , PREDICT: home_win (1.48, 1.44) , ACTUAL: 12.0%
2122 GW11 WHU 3 - 2 LIV , PREDICT: away_win (1.2, 1.93) , ACTUAL: 2.3%
2122 GW12 LEI 0 - 3 CHE , PREDICT: away_win (1.4, 1.44) , ACTUAL: 2.9%
2122 GW12 AVL 2 - 0 BHA , PREDICT: home_win (1.28, 0.99) , ACTUAL: 8.4%
2122 GW12 BUR 3 - 3 CRY , PREDICT: home_win (1.21, 1.04) , ACTUAL: 0.6%
2122 GW12 NEW 3 - 3 BRE , PREDICT: home_win (1.41, 1.07) , ACTUAL: 0.8%
2122 GW12 NOR 2 - 1 SOU , PREDICT: away_win (1.21, 1.37) , ACTUAL: 7.5%
2122 GW12 WAT 4 - 1 MUN , PREDICT: away_win (0.94, 1.67) , ACTUAL: 0.4%
2122 GW12 WOL 1 - 0 WHU , PREDICT: home_win (1.47, 1.17) , ACTUAL: 9.8%
2122 GW12 LIV 4 - 0 ARS , PREDICT: home_win (1.96, 1.17) , ACTUAL: 2.7%
2122 GW12 MCI 3 - 0 EVE , PREDICT: home_win (2.35, 0.86) , ACTUAL: 8.8%
2122 GW12 TOT 2 - 1 LEE , PREDICT: home_win (2.1, 0.99) , ACTUAL: 9.8%
2122 GW13 ARS 2 - 0 NEW , PREDICT: home_win (2.0, 0.89) , ACTUAL: 11.0%
2122 GW13 CRY 1 - 2 AVL , PREDICT: home_win (1.34, 1.08) , ACTUAL:

2122 GW25 WAT 0 - 2 BHA , PREDICT: home_win (1.25, 1.06) , ACTUAL: 5.6%
2122 GW25 NOR 0 - 4 MCI , PREDICT: away_win (0.81, 2.48) , ACTUAL: 6.0%
2122 GW25 BUR 0 - 1 LIV , PREDICT: away_win (0.88, 1.84) , ACTUAL: 11.5%
2122 GW25 NEW 1 - 0 AVL , PREDICT: home_win (1.36, 1.08) , ACTUAL: 11.1%
2122 GW25 TOT 0 - 2 WOL , PREDICT: home_win (1.89, 0.88) , ACTUAL: 2.4%
2122 GW25 LEI 2 - 2 WHU , PREDICT: home_win (1.79, 1.19) , ACTUAL: 5.7%
2122 GW25 MUN 2 - 0 BHA , PREDICT: home_win (1.83, 0.71) , ACTUAL: 13.2%
2122 GW26 WHU 1 - 1 NEW , PREDICT: home_win (1.64, 1.08) , ACTUAL: 12.2%
2122 GW26 ARS 2 - 1 BRE , PREDICT: home_win (2.11, 0.88) , ACTUAL: 9.7%
2122 GW26 AVL 0 - 1 WAT , PREDICT: home_win (1.43, 1.06) , ACTUAL: 8.1%
2122 GW26 BHA 0 - 3 BUR , PREDICT: home_win (1.14, 0.96) , ACTUAL: 1.8%
2122 GW26 CRY 0 - 1 CHE , PREDICT: away_win (1.03, 1.61) , ACTUAL: 10.9%
2122 GW26 LIV 3 - 1 NOR , PREDICT: home_win (2.74, 0.71) , ACTUAL: 7.6%
2122 GW26 SOU 2 - 0 EVE , PREDICT: away_win (1.27, 1.28) , 

2122 GW36 BHA 4 - 0 MUN , PREDICT: away_win (0.87, 1.5) , ACTUAL: 0.2%
2122 GW36 LIV 1 - 1 TOT , PREDICT: home_win (1.74, 1.3) , ACTUAL: 11.3%
2122 GW36 ARS 2 - 1 LEE , PREDICT: home_win (1.94, 1.12) , ACTUAL: 9.8%
2122 GW36 LEI 1 - 2 EVE , PREDICT: home_win (1.6, 1.13) , ACTUAL: 6.7%
2122 GW36 NOR 0 - 4 WHU , PREDICT: away_win (1.2, 1.58) , ACTUAL: 1.6%
2122 GW36 MCI 5 - 0 NEW , PREDICT: home_win (2.63, 0.73) , ACTUAL: 3.8%
2122 GW36 AVL 1 - 2 LIV , PREDICT: away_win (0.96, 1.96) , ACTUAL: 9.8%
2122 GW36 LEE 0 - 3 CHE , PREDICT: away_win (1.3, 1.57) , ACTUAL: 3.7%
2122 GW36 LEI 3 - 0 NOR , PREDICT: home_win (2.1, 0.87) , ACTUAL: 8.0%
2122 GW36 WAT 0 - 0 EVE , PREDICT: away_win (1.15, 1.39) , ACTUAL: 8.7%
2122 GW36 WOL 1 - 5 MCI , PREDICT: away_win (0.99, 1.86) , ACTUAL: 1.1%
2122 GW36 TOT 3 - 0 ARS , PREDICT: home_win (1.79, 1.18) , ACTUAL: 4.9%
2122 GW37 TOT 1 - 0 BUR , PREDICT: home_win (2.06, 0.73) , ACTUAL: 12.2%
2122 GW37 AVL 1 - 1 CRY , PREDICT: home_win (1.32, 1.11) , ACTUAL: 1

# Model 2

Updated model with data from each week and predict next

In [ ]:
full_df = []

for week in range(1,max_week+1):
    print (week)

    df = run_match_prediction(week,season, max_goals,0)
    
    full_df.append(df)
    
one_week_prediction = pd.concat(full_df)


1
Fitting team model...


sample: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:18<00:00, 80.76it/s, 127 steps of size 4.15e-02. acc. prob=0.93]


Adding BRE to team model with covariates
2122 GW1 BRE 2 - 0 ARS , PREDICT: away_win (1.08, 1.71) , ACTUAL: 3.6%
2122 GW1 MUN 5 - 1 LEE , PREDICT: home_win (1.83, 0.99) , ACTUAL: 1.0%
2122 GW1 BUR 1 - 2 BHA , PREDICT: home_win (1.18, 0.93) , ACTUAL: 6.2%
2122 GW1 CHE 3 - 0 CRY , PREDICT: home_win (1.98, 0.85) , ACTUAL: 7.6%
2122 GW1 EVE 3 - 1 SOU , PREDICT: home_win (1.6, 1.04) , ACTUAL: 5.0%
2122 GW1 LEI 1 - 0 WOL , PREDICT: home_win (1.57, 1.07) , ACTUAL: 10.6%
2122 GW1 WAT 3 - 2 AVL , PREDICT: home_win (1.31, 1.17) , ACTUAL: 2.1%
2122 GW1 NOR 0 - 3 LIV , PREDICT: away_win (0.88, 2.26) , ACTUAL: 8.4%
2122 GW1 NEW 2 - 4 WHU , PREDICT: away_win (1.34, 1.35) , ACTUAL: 0.8%
2122 GW1 TOT 1 - 0 MCI , PREDICT: away_win (1.46, 1.56) , ACTUAL: 6.5%
2
Fitting team model...


sample: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:19<00:00, 76.03it/s, 127 steps of size 3.70e-02. acc. prob=0.94]


2122 GW2 LIV 2 - 0 BUR , PREDICT: home_win (2.27, 0.72) , ACTUAL: 12.9%
2122 GW2 AVL 2 - 0 NEW , PREDICT: home_win (1.33, 1.11) , ACTUAL: 7.7%
2122 GW2 CRY 0 - 0 BRE , PREDICT: home_win (1.46, 1.01) , ACTUAL: 9.2%
2122 GW2 LEE 2 - 2 EVE , PREDICT: home_win (1.49, 1.27) , ACTUAL: 5.6%
2122 GW2 MCI 5 - 0 NOR , PREDICT: home_win (3.04, 0.65) , ACTUAL: 5.7%
2122 GW2 BHA 2 - 0 WAT , PREDICT: home_win (1.31, 1.02) , ACTUAL: 8.4%
2122 GW2 SOU 1 - 1 MUN , PREDICT: away_win (1.04, 1.56) , ACTUAL: 12.6%
2122 GW2 WOL 0 - 1 TOT , PREDICT: away_win (1.06, 1.54) , ACTUAL: 10.8%
2122 GW2 ARS 0 - 2 CHE , PREDICT: home_win (1.56, 1.34) , ACTUAL: 5.0%
2122 GW2 WHU 4 - 1 LEI , PREDICT: away_win (1.46, 1.48) , ACTUAL: 1.5%
3
Fitting team model...


...

2122 GW20 CRY 3 - 0 NOR , PREDICT: home_win (1.61, 0.91) , ACTUAL: 5.7%
2122 GW20 SOU 1 - 1 TOT , PREDICT: away_win (1.04, 1.75) , ACTUAL: 11.7%
2122 GW20 WAT 1 - 4 WHU , PREDICT: away_win (1.31, 1.5) , ACTUAL: 1.7%
2122 GW20 LEI 1 - 0 LIV , PREDICT: away_win (1.31, 1.8) , ACTUAL: 5.3%
2122 GW20 CHE 1 - 1 BHA , PREDICT: home_win (1.92, 0.73) , ACTUAL: 10.4%
2122 GW20 BRE 0 - 1 MCI , PREDICT: away_win (0.82, 2.39) , ACTUAL: 9.2%
2122 GW20 MUN 3 - 1 BUR , PREDICT: home_win (1.8, 0.73) , ACTUAL: 5.6%
21
Fitting team model...


sample: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:21<00:00, 69.90it/s, 127 steps of size 3.56e-02. acc. prob=0.93]


2122 GW21 ARS 1 - 2 MCI , PREDICT: away_win (1.29, 1.79) , ACTUAL: 9.4%
2122 GW21 WAT 0 - 1 TOT , PREDICT: away_win (0.96, 1.92) , ACTUAL: 10.3%
2122 GW21 CRY 2 - 3 WHU , PREDICT: away_win (1.34, 1.37) , ACTUAL: 2.6%
2122 GW21 BRE 2 - 1 AVL , PREDICT: home_win (1.28, 1.21) , ACTUAL: 8.1%
2122 GW21 EVE 2 - 3 BHA , PREDICT: home_win (1.51, 0.88) , ACTUAL: 1.2%
2122 GW21 LEE 3 - 1 BUR , PREDICT: home_win (1.52, 1.01) , ACTUAL: 4.7%
2122 GW21 CHE 2 - 2 LIV , PREDICT: away_win (1.46, 1.51) , ACTUAL: 6.2%
2122 GW21 MUN 0 - 1 WOL , PREDICT: home_win (1.57, 0.84) , ACTUAL: 6.9%
2122 GW21 SOU 4 - 1 BRE , PREDICT: home_win (1.56, 1.02) , ACTUAL: 1.9%
2122 GW21 WHU 2 - 0 NOR , PREDICT: home_win (2.03, 0.9) , ACTUAL: 11.0%
22
Fitting team model...


sample: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:22<00:00, 68.02it/s, 127 steps of size 4.31e-02. acc. prob=0.90]


2122 GW22 BHA 1 - 1 CRY , PREDICT: home_win (1.19, 1.06) , ACTUAL: 13.9%
2122 GW22 MCI 1 - 0 CHE , PREDICT: home_win (2.04, 1.06) , ACTUAL: 8.7%
2122 GW22 NEW 1 - 1 WAT , PREDICT: home_win (1.47, 1.09) , ACTUAL: 13.0%
2122 GW22 NOR 2 - 1 EVE , PREDICT: away_win (1.0, 1.56) , ACTUAL: 6.0%
2122 GW22 WOL 3 - 1 SOU , PREDICT: home_win (1.39, 0.96) , ACTUAL: 4.1%
2122 GW22 AVL 2 - 2 MUN , PREDICT: away_win (0.98, 1.6) , ACTUAL: 4.6%
2122 GW22 LIV 3 - 0 BRE , PREDICT: home_win (2.68, 0.74) , ACTUAL: 10.5%
2122 GW22 WHU 2 - 3 LEE , PREDICT: home_win (1.72, 1.32) , ACTUAL: 2.7%
2122 GW22 BHA 1 - 1 CHE , PREDICT: away_win (0.9, 1.56) , ACTUAL: 12.5%
2122 GW22 LEI 2 - 3 TOT , PREDICT: away_win (1.33, 1.57) , ACTUAL: 3.1%
2122 GW22 BRE 1 - 3 MUN , PREDICT: away_win (0.92, 1.73) , ACTUAL: 5.5%
23
Fitting team model...


# Results 

Compare effect of training data on model predictions. Doesnt seem to be much difference.

In [ ]:
one_to_38_week_prediction.head()


In [ ]:
one_week_prediction.head()

In [ ]:
one_to_38_week_prediction.tail()


In [ ]:
one_week_prediction.tail()

In [ ]:
make_scatter_error_plot(one_to_38_week_prediction,'probability_result')
make_scatter_error_plot(one_week_prediction,'probability_result','r')

In [ ]:
make_scatter_error_plot(one_to_38_week_prediction,'diff_score_mean')
make_scatter_error_plot(one_week_prediction,'diff_score_mean','r')

In [ ]:
make_scatter_error_plot(one_to_38_week_prediction,'result_correct')
make_scatter_error_plot(one_week_prediction,'result_correct','r')

In [ ]:
make_scatter_error_plot(one_to_38_week_prediction,'home_score_correct')
make_scatter_error_plot(one_week_prediction,'home_score_correct','r')


In [ ]:
make_scatter_error_plot(one_to_38_week_prediction,'away_score_correct')
make_scatter_error_plot(one_week_prediction,'away_score_correct','r')
